In [ ]:
import pandas as pd

#en_hi_tweets = open("/content/drive/MyDrive/nlp_project/datasets/POStag_dataset/POS_Hindi_English_Code_Mixed_Tweets.tsv", "r").read()
#data2 = open("/content/drive/MyDrive/nlp_project/datasets/POStag_dataset/data2.txt", "r").read()
#whatsapp = open("/content/drive/MyDrive/nlp_project/datasets/POStag_dataset/WaData.txt", "r").read()
pk = open("/content/drive/MyDrive/nlp_project/datasets/POStag_dataset/pk_modi.txt", "r").read()
en_tags = open("/content/drive/MyDrive/nlp_project/datasets/POStag_dataset/en_tags.txt", "r").read()

In [ ]:
def token_lookup():
    token_dict = {".":"<PERIOD>", ",": "<COMMA>", ";":"<SEMICOLON>", ":":"<COLON>", "\"":"<QOUTATION>", "'":"<QOUTATION>", "!":"<EXCLAMATION>", "?":"<QUESTION>", 
                  "(": "<LEFTPARAN>", ")": "<RIGHTPARAN>", "{":"LEFTBRACE", "}":"<RIGHTBRACE>", "[":"<LEFTBRACKET>", "]":"<RIGHTBRACKET>", "--":"<DASH>", "-":"<HYPHEN>",
                  #"\n":"<RETURN>"
                  }
    return token_dict
def create_sentences(lines):
    sentences = []
    for line in lines:
        rows = line.split('\n')
        sent = ['<BOS>']
        for x in rows:
            x = x.split('\t')
            if len(x)>2 and x[0].isalpha():
                sent.append(x[0]+"_"+x[2])
            #sent.append("_".join(x.split('\t')))
        sent.append("<EOS>")
        sent = ' '.join(sent)
        sentences.append(sent)
    return sentences
token_dict = token_lookup()

for key, token in token_dict.items():
    pk = pk.replace(key, '{}'.format(token))
    en_tags = en_tags.replace(key, '{}'.format(token))


#lines = en_hi_tweets.split("\n\t\t\n")
#lines = data2.split("\n\n")
#lines = whatsapp.split("\n\n")
lines_pk = create_sentences(pk.split("\n\n"))
lines_en_tags = create_sentences(en_tags.split("\n\n"))
lines_pk = pd.DataFrame(lines_pk, columns=["sentences"])
lines_en_tags = pd.DataFrame(lines_en_tags, columns=["sentences"])

en_mix = lines_pk #pd.concat([lines_en_tags, lines_pk], axis=0)
#en_mix = en_mix.sample(frac = 1)
en_mix.head()
#print(len(sentences))

,sentences
0,<BOS> bharat_G_N me_G_PRP ajaad_G_V he_G_V lek...
1,<BOS> digital_G_J indiaa_G_N <EOS>
2,<BOS> enn_G_J sab_G_N ko_PSP ban_G_N <EOS>
3,<BOS> karna_G_N cha_G_V he_G_V ye_G_PRP jab_G_...
4,<BOS> part_G_N time_G_N jop_G_N dene_G_N ka_PS...


In [ ]:
def create_lookup_tables(text_df):
    vocab = set()
    for line in text_df['sentences']:
        vocab.update(set(line.split()))
    #vocab = set(text)
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return (vocab_to_int, int_to_vocab)

vocab_to_int, int_to_vocab = create_lookup_tables(en_mix)

train_int_text = []

for line in en_mix['sentences']:
    train_int_text.append([vocab_to_int[word] for word in line.split()])
print(len(vocab_to_int))
train_int_text = [item for sublist in train_int_text for item in sublist]
print(train_int_text[:1000])  

24636
[10326, 12074, 21541, 7202, 12738, 20950, 14760, 6496, 19923, 21629, 24276, 17817, 3346, 22390, 24082, 10002, 24609, 17541, 19141, 6248, 17920, 12074, 4958, 16714, 22760, 1726, 15275, 11896, 12738, 12824, 1118, 17541, 12738, 7064, 18756, 445, 19765, 11424, 4958, 122, 12034, 4958, 11385, 903, 12232, 12139, 18200, 6734, 17211, 12232, 7181, 14804, 12726, 3126, 21092, 11744, 17862, 21524, 2844, 15894, 12232, 11899, 10326, 7938, 20521, 11899, 10326, 6176, 6570, 4958, 1109, 11899, 10326, 18232, 15002, 12738, 10346, 3346, 12805, 23860, 10874, 2066, 16714, 11452, 6588, 3975, 7756, 12838, 23283, 21524, 6338, 2926, 11972, 5811, 3322, 665, 18401, 21524, 21348, 5810, 1789, 15352, 19231, 24143, 5686, 18631, 445, 4958, 8964, 6261, 3263, 2126, 16342, 24059, 5920, 21004, 20579, 17936, 597, 5608, 16348, 3975, 8467, 13947, 24059, 19879, 5608, 17569, 9493, 21375, 1789, 17920, 14529, 20809, 1015, 13865, 17818, 20496, 11748, 3975, 4685, 14484, 10507, 6338, 5753, 17329, 4958, 16714, 11452, 20874, 1707

In [ ]:
#import helper

#data_dir = './brown.txt'

#helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

from distutils.version import LooseVersion
import warnings
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import tensorflow as tf2
#import helper
import numpy as np

#int_text, val_int_text, test_int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

def get_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learn_rate = tf.placeholder(tf.float32, name='learn_rate')
    return (inputs, targets, learn_rate)
    
def get_init_cell(batch_size, rnn_size):
    def build_cell(lstm_size):
        # Use a basic LSTM cell
        lstm = tf2.compat.v1.nn.rnn_cell.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf2.compat.v1.nn.rnn_cell.DropoutWrapper(lstm)#, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    num_layers = 2
    cell = tf2.compat.v1.nn.rnn_cell.MultiRNNCell([build_cell(rnn_size) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, dtype=tf.float32)
    initial_state = tf.identity(initial_state, name="initial_state")
    return (cell, initial_state)
    
def get_embed(input_data, vocab_size, embed_dim):
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed
    
def build_rnn(cell, inputs):
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)#,initial_state=initial_state)
    final_state = tf.identity(final_state, name="final_state")
    return outputs, final_state
    
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    
    logits = tf2.compat.v1.layers.dense(outputs, vocab_size, activation=None)
    return logits, final_state
    
def get_batches(int_text, batch_size, seq_length):
    words_per_batch = batch_size * seq_length
    num_batch = len(int_text)//words_per_batch
    
    input_text = np.array(int_text[:(num_batch*words_per_batch)])
    target_text = np.array(int_text[1:(num_batch * words_per_batch) + 1])

    target_text[-1] = input_text[0]
    
    x_batches = np.split(input_text.reshape(batch_size, -1), num_batch, 1)
    y_batches = np.split(target_text.reshape(batch_size, -1), num_batch, 1)
    
    batches = np.array(list(zip(x_batches, y_batches)))
    return batches
    
# Number of Epochs
num_epochs = 60
# Batch Size
batch_size = 100
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 10
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 50

save_dir = '/content/drive/MyDrive/nlp_project/models/model_2_postag_en_hi_t12/save'

batches = get_batches(train_int_text, batch_size, seq_length)

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
!pip install tensorflow_addons
import tensorflow_addons as tfa


train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = tfa.seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
    
#batches = get_batches(train_int_text, batch_size, seq_length)
#train_graph = tf.Graph()
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')
    
# Save parameters for checkpoint
#helper.save_params((seq_length, save_dir))

     |████████████████████████████████| 706kB 8.1MB/s 
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:702: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/pyth

Epoch   0 Batch    0/208   train_loss = 10.112
Epoch   0 Batch   50/208   train_loss = 8.706
Epoch   0 Batch  100/208   train_loss = 7.910
Epoch   0 Batch  150/208   train_loss = 7.864
Epoch   0 Batch  200/208   train_loss = 7.798
Epoch   1 Batch   42/208   train_loss = 7.531
Epoch   1 Batch   92/208   train_loss = 7.401
Epoch   1 Batch  142/208   train_loss = 7.471
Epoch   1 Batch  192/208   train_loss = 7.456
Epoch   2 Batch   34/208   train_loss = 7.214
Epoch   2 Batch   84/208   train_loss = 7.195
Epoch   2 Batch  134/208   train_loss = 7.059
Epoch   2 Batch  184/208   train_loss = 7.214
Epoch   3 Batch   26/208   train_loss = 7.097
Epoch   3 Batch   76/208   train_loss = 6.840
Epoch   3 Batch  126/208   train_loss = 7.055
Epoch   3 Batch  176/208   train_loss = 6.906
Epoch   4 Batch   18/208   train_loss = 6.838
Epoch   4 Batch   68/208   train_loss = 6.691
Epoch   4 Batch  118/208   train_loss = 6.693
Epoch   4 Batch  168/208   train_loss = 6.769
Epoch   5 Batch   10/208   train_

In [ ]:
import tensorflow as tf
import numpy as np
#import helper

def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    index = np.where(probabilities == np.max(probabilities))[-1][0]

    return int_to_vocab[index]

#_,_,_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = seq_length, '/content/drive/MyDrive/nlp_project/models/model_2_postag_en_hi_t12/save'
token_dict = token_lookup()
def get_tensors(loaded_graph):
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    prob_tensor = loaded_graph.get_tensor_by_name("probs:0")

    return input_tensor, initial_state_tensor, final_state_tensor, prob_tensor
    
#test_text = [int_to_vocab[word] for word in test_int_text]
#test_text = " ".join(test_text)
#test_sent_text = test_text.split("<period>")

'''
test_sentences = input("Enter sentence:")
token_dict = token_lookup()
for key, token in token_dict.items():
	test_sentences = test_sentences.replace(key, ' {} '.format(token))
test_sentences = test_sentences.lower()
test_sentences = test_sentences.split("<period>")
'''
#all_perp = []

loaded_graph = tf.Graph()
with tf.compat.v1.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.compat.v1.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)
    #print(final_state)
    # Sentences generation setup
    gen_length = 4000
    prime_word = "digital_G_J" #"Chalo_hi_PRT"#"Subah_hi_PART" #'यही' #'moe_szyslak'
    gen_sentences = [prime_word]
            
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])
          
        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})

        #pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)

    # Remove tokens
    script = []
    for word in gen_sentences:
        script.append(word.split("_")[0])
    tv_script = ' '.join(script)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    #tv_script = tv_script.replace('<BOS>', '')
    #tv_script = tv_script.replace('<EOS>', '\n')    
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/nlp_project/models/model_2_postag_en_hi_t12/save
digital darj times rd jeeo hojata lagti underestimate sunta curroption hamara govrment regal vadiyan gaurd opps sunta dikhaunga gndi agai apna faltu osne bakwaas ukhadlo copy opps sharm bajane medi padhne audience deya pather bhaut bareme opps km highjack jindabad fogging bhi nashe kabki among z beithe cutiya shocking kabki publicity z aa outstanding pehna padak tie yoy jito lotery corore currpt latt cp cp otherwise mani otherwise jaen interval bhojan haha yoy kah picnic girni adani sawraj khola sou jhonk pichla mafi reports z handi thiz neither hear saboot indutry next side tenat dekhti mass vyakti columns gye blles bhooke kamayegi bhi inta gutter hazaar itni inta mandbuddhi aaam clay aaditya bhooke hear i adityanath indiraji kuvh pahuchi jimmy failayega tenat z darj kamayegi medhak faad mau nly pehen naxal adityanath hides bles morning chaddi mood purne jakir columns bhri